# Data-Centric NLP 대회: 주제 분류 프로젝트

## Load Libraries

In [28]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

## Set Hyperparameters

In [29]:
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [30]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [31]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../data/augmented/combined_rows_aug_v4.csv') # processed된 파일까지 경로 
TEST_DIR = os.path.join(BASE_DIR, '../data/raw/test.csv')
train_name = os.path.splitext(os.path.basename(DATA_DIR))[0]  # processed된 파일 이름 추출
OUTPUT_DIR = os.path.join(BASE_DIR, f'../output/{train_name}') # 해당 파일 이름으로 output 폴더 생성

## Load Tokenizer and Model

In [32]:
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to(DEVICE)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define Dataset

In [33]:
data = pd.read_csv(DATA_DIR)
dataset_train, dataset_valid = train_test_split(data, test_size=0.1, random_state=SEED)

In [34]:
class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):
        input_texts = data['text']
        targets = data['target']
        self.inputs = []; self.labels = []
        for text, label in zip(input_texts, targets):
            tokenized_input = tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')
            self.inputs.append(tokenized_input)
            self.labels.append(torch.tensor(label))

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(0),
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(0),
            'labels': self.labels[idx].squeeze(0)
        }

    def __len__(self):
        return len(self.labels)

In [35]:
data_train = BERTDataset(dataset_train, tokenizer)
data_valid = BERTDataset(dataset_valid, tokenizer)

In [36]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Define Metric

In [37]:
f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')


## Train Model

In [38]:
### for wandb setting
#os.environ['WANDB_DISABLED'] = 'true'

In [39]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=10,
    eval_steps=10,
    save_steps=100,
    save_total_limit=2,
    learning_rate= 3e-05,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED
)

In [40]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [41]:
trainer.train()

Step,Training Loss,Validation Loss,F1
10,1.937200,1.762350,0.411507
20,1.729300,1.523712,0.604673
30,1.414500,1.205333,0.688194
40,1.094200,0.968727,0.741150
50,0.950000,0.820865,0.759582
60,0.789100,0.749108,0.770445
70,0.640500,0.749939,0.750443
80,0.748100,0.652005,0.801527
90,0.708500,0.614651,0.815283
100,0.640000,0.609059,0.798876


TrainOutput(global_step=344, training_loss=0.6252312233974767, metrics={'train_runtime': 498.3, 'train_samples_per_second': 22.039, 'train_steps_per_second': 0.69, 'total_flos': 2889615327590400.0, 'train_loss': 0.6252312233974767, 'epoch': 2.0})

## Evaluate Model

In [42]:
dataset_test = pd.read_csv(TEST_DIR)

In [43]:
model.eval()
preds = []

for idx, sample in tqdm(dataset_test.iterrows(), total=len(dataset_test), desc="Evaluating"):
    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        preds.extend(pred)

Evaluating: 100%|██████████| 30000/30000 [05:44<00:00, 87.11it/s]


In [44]:
dataset_test['target'] = preds
dataset_test.to_csv(os.path.join(OUTPUT_DIR, 'output_v4_lr3.csv'), index=False) # processed된 파일 이름 폴더에 결과 저장